# <img src='https://raw.githubusercontent.com/huseidon/RCloneColab/refs/heads/main/img/title_jdownloader.png' height="35" alt="JDownloader"/>

In [ ]:
#@markdown <h3>⬅️ Install JDownloader</h3>

import os, uuid, re, IPython
import ipywidgets as widgets

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output

Email = widgets.Text(placeholder="*Required", description="Email:")
Password = widgets.Text(placeholder="*Required", description="Password:")
Device = widgets.Text(placeholder="Optional", description="Name:")
SavePath = widgets.Dropdown(value="/content/Downloads", options=["/content", "/content/Downloads"], description="Save Path:")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html

def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPath():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePath.options = ["/content", "/content/Downloads"]

def LoginForm():
  clear_output(wait=True)
  Email.value = ""
  Password.value = ""
  Device.value = "RCloneColab"
  RefreshPath()
  display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If you don't have an account yet, please register <a href=\"https://my.jdownloader.org/login.html#register\" target=\"_blank\">here</a>.</h3>"), HTML("<br>"), Email, Password, Device, SavePath, MakeButton("Refresh", RefreshPath, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("<br>"), MakeButton("Login", CheckLogin, "info"))
  if os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
    display(MakeButton("Cancel", Show, "danger"))

def RestartForm():
  clear_output(wait=True)
  display(MakeLabel("Restart Confirm?", ""), MakeButton("Confirm", Restart, "danger"), MakeButton("Cancel", Show, "warning"))

def ExitForm():
  clear_output(wait=True)
  display(MakeLabel("Exit Confirm?", ""), MakeButton("Confirm", Exit, "danger"), MakeButton("Cancel", Show, "warning"))

def CheckLogin():
  try:
    if not Email.value.strip():
      ERROR = "Email field is empty."
      THROW_ERROR
    if not "@" in Email.value and not "." in Email.value:
      ERROR = "Email is an incorrect format."
      THROW_ERROR
    if not Password.value.strip():
      ERROR = "Password field is empty."
      THROW_ERROR
    if not bool(re.match("^[a-zA-Z0-9]+$", Device.value)) and Device.value.strip():
      ERROR = "Only alphanumeric are allowed for the device name."
      THROW_ERROR
    Login()
  except:
    print(ERROR)

def Login():
  clear_output(wait=True)
  if SavePath.value == "/content":
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  elif SavePath.value == "/content/Downloads":
    get_ipython().system_raw("mkdir -p -m 666 /content/Downloads")
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content/Downloads\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  else:
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"" + SavePath.value + "\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  if Device.value.strip() == "":
    Device.value = Email.value
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("echo '{\"email\" : \"'" + Email.value + "'\", \"password\" : \"'" + Password.value + "'\", \"devicename\" : \"'" + Device.value + "'\", \"directconnectmode\" : \"LAN\"}' > /root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()

def Restart():
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()

def Exit():
  get_ipython().system_raw("pkill -9 -e -f java")
  clear_output(wait=True)
  display(MakeButton("Start", Restart, "info"))

def Show():
  clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Login to the MyJDownloader WebUI by clicking <a href=\"https://my.jdownloader.org/\" target=\"_blank\">here</a>.</h3>"), HTML("<h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If the server didn't showup in 30 sec. please re-login.</h4>"), HTML("<br>"), MakeButton("Re-Login", LoginForm, "info"), MakeButton("Restart", RestartForm, "warning"), MakeButton("Exit", ExitForm, "danger"))


if not os.path.isfile("/root/.JDownloader/JDownloader.jar"):
  clear_output(wait=True)
  display(MakeLabel("Installing in Progress", "warning"))
  get_ipython().system_raw("rm -rf /content/sample_data/ && apt install openjdk-8-jre-headless -qq -y && mkdir -p -m 666 /root/.JDownloader/libs && wget -q http://installer.jdownloader.org/JDownloader.jar -O /root/.JDownloader/JDownloader.jar && java -jar /root/.JDownloader/JDownloader.jar -norestart -h && wget -q https://raw.githubusercontent.com/huseidon/RCloneColab/refs/heads/main/res/sevenzipjbinding1509.jar -O /root/.JDownloader/libs/sevenzipjbinding1509.jar && wget -q https://raw.githubusercontent.com/huseidon/RCloneColab/refs/heads/main/res/sevenzipjbinding1509Linux.jar -O /root/.JDownloader/libs/sevenzipjbinding1509Linux.jar")
  LoginForm()
elif not os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
  LoginForm()
else:
  Show()

# <img src='https://raw.githubusercontent.com/huseidon/RCloneColab/refs/heads/main/img/title_rclone.png' height="35" alt="RcloneLab"/>

In [ ]:
#@markdown <h3>⬅️ Install RClone</h3>
Beta = False
Setup_Time_Zone = False

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Upload your config file from your computer.(rclone.conf)</h2><br>"))
    UploadConfig = files.upload().keys()
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn):
          get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.rclone.conf && chmod 666 /root/.rclone.conf")
          if Setup_Time_Zone:
            !sudo dpkg-reconfigure tzdata
          if os.path.isfile("/usr/bin/rclone"):
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            if Beta:
              get_ipython().system_raw("rm -rf /content/sample_data/ && curl -s https://rclone.org/install.sh | sudo bash -s beta")
            else:
              get_ipython().system_raw("rm -rf /content/sample_data/ && curl -s https://rclone.org/install.sh | sudo bash")
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          clear_output(wait=True)
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

In [ ]:
#@markdown <h3>⬅️ Run RcloneColab</h3>
# ============================= FORM ============================= #
Mode = "Move" #@param ["Config", "Copy", "Move", "Mount", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash"]
Compare = "Only Size" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Source = "Downloads/" #@param {type:"string"}
Destination = "gdrive:" #@param {type:"string"}
Transfers = 1 #@param {type:"slider", min:1, max:20, step:1}
Checkers = 10
# ================================================================ #
Simple_Ouput = True
Skip_files_that_are_newer_on_the_destination = False
Skip_all_files_that_exist = False
Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False
Large_amount_of_files_optimization = False
Google_Drive_optimization = True #@param {type:"boolean"}
Google_Drive_server_side_copy = False
Dry_Run = False #@param {type:"boolean"}
Output_Log_File = "OFF"
    # Other parameters for output log file OFF , NOTICE , INFO , ERROR , DEBUG #
Extra_Arguments = "" #@param {type:"string"}
# ================================================================ #
Sync_Mode = "Delete after transfering"
    # #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"] #
Track_Renames = False
# ================================================================ #
Deduplicate_Mode = "Interactive"
      # "Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename" #
Deduplicate_Use_Trash = False
# ================================================================ #

import os, psutil; from IPython.display import HTML, clear_output; from IPython.utils.io import ask_yes_no

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):
            return True
          else:
            pass
      else:
        pass
    except:
      continue

### Variable Declaration

# Optimized for Google Colaboratory
if Google_Drive_optimization:
  bufferC = "--buffer-size 128M"
else:
  bufferC = "--buffer-size 256M"

if Compare == "Size & Checksum":
  compareC = "-c"
elif Compare == "Only Mod-Time":
  compareC = "--ignore-size"
elif Compare == "Only Size":
  compareC = "--size-only"
elif Compare == "Only Checksum":
  compareC = "-c --ignore-size"
else:
  compareC = ""

transfersC = "--transfers "+str(Transfers)
checkersC = "--checkers "+str(Checkers)

if Skip_files_that_are_newer_on_the_destination:
  skipnewC = "-u"
else:
  skipnewC = ""

if Skip_all_files_that_exist:
  skipexistC = "--ignore-existing"
else:
  skipexistC = ""

if Do_not_cross_filesystem_boundaries:
  nocrossfilesystemC = "--one-file-system"
else:
  nocrossfilesystemC = ""

if Do_not_update_modtime_if_files_are_identical:
  noupdatemodtimeC = "--no-update-modtime"
else:
  noupdatemodtimeC = ""

if Large_amount_of_files_optimization:
  filesoptimizeC = "--fast-list"
else:
  filesoptimizeC = ""

if Google_Drive_optimization:
  driveoptimizeC = "--drive-chunk-size 128M --drive-acknowledge-abuse --drive-keep-revision-forever"
else:
  driveoptimizeC = ""

if Google_Drive_server_side_copy:
  drivecopyC = "--drive-server-side-across-configs=true"
elif Google_Drive_optimization:
  drivecopyC = "--disable copy"

if Dry_Run:
  dryrunC = "-n"
else:
  dryrunC = ""

if Output_Log_File != "OFF":
  statsC = "--log-file=/root/.rclone_log/rclone_log.txt"
else:
  if Simple_Ouput:
    statsC = "-v --stats-one-line --stats=5s"
  else:
    statsC = "-v --stats=5s"

if Output_Log_File == "INFO":
  loglevelC = "--log-level INFO"
elif Output_Log_File == "ERROR":
  loglevelC = "--log-level ERROR"
elif Output_Log_File == "DEBUG":
  loglevelC = "--log-level DEBUG"
else:
  loglevelC = ""

if Sync_Mode == "Delete during transfer":
  syncmodeC = "--delete-during"
elif Sync_Mode == "Delete before transfering":
  syncmodeC = "--delete-before"
elif Sync_Mode == "Delete after transfering":
  syncmodeC = "--delete-after"

if Track_Renames:
  trackrenamesC = "--track-renames"
else:
  trackrenamesC = ""

if Deduplicate_Mode == "Interactive":
  deduplicateC = "interactive"
elif Deduplicate_Mode == "Skip":
  deduplicateC = "skip"
elif Deduplicate_Mode == "First":
  deduplicateC = "first"
elif Deduplicate_Mode == "Newest":
  deduplicateC = "newest"
elif Deduplicate_Mode == "Oldest":
  deduplicateC = "oldest"
elif Deduplicate_Mode == "Largest":
  deduplicateC = "largest"
elif Deduplicate_Mode == "Rename":
  deduplicateC = "rename"

if Deduplicate_Use_Trash:
  deduplicatetrashC = ""
else:
  deduplicatetrashC = "--drive-use-trash=false"

### rclone Execution

if Output_Log_File != "OFF" and Mode != "Config":
  get_ipython().system_raw("mkdir -p -m 666 /root/.rclone_log/")
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enables, rclone will not output log through the terminal, please wait until finished.</h2><br></center>"))

if Mode == "Config":
  !rclone --config=/root/.rclone.conf config
elif Mode == "Copy":
  !rclone --config=/root/.rclone.conf copy "$Source" "$Destination" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Move":
  !rclone --config=/root/.rclone.conf move "$Source" "$Destination" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Mount":
  try:
    if not CheckProcess("rclone", "/content/" + Source.split(":")[0] + "/"):
      if not os.path.exists("/content/" + Source.split(":")[0] + "/"):
        get_ipython().system_raw("mkdir -p -m 666 '/content/" + Source.split(":")[0] + "/'")
      get_ipython().system_raw("rclone --config=/root/.rclone.conf mount '$Source' '/content/" + Source.split(":")[0] + "/' $loglevelC $noupdatemodtimeC $bufferC $filesoptimizeC --drive-chunk-size 128M --drive-acknowledge-abuse --drive-keep-revision-forever $Extra_Arguments &")
  except:
    pass
elif Mode == "Sync":
  !rclone --config=/root/.rclone.conf sync "$Source" "$Destination" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Checker":
  !rclone --config=/root/.rclone.conf check "$Source" "$Destination" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Deduplicate":
  !rclone --config=/root/.rclone.conf dedupe "$Source" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Remove Empty Directories":
  !rclone --config=/root/.rclone.conf rmdirs "$Source" $statsC $loglevelC $dryrunC $Extra_Arguments
elif Mode == "Empty Trash":
  !rclone --config=/root/.rclone.conf cleanup "$Source" $statsC $loglevelC $dryrunC $Extra_Arguments

### Log Output

if Output_Log_File != "OFF" and Mode != "Config" and os.path.isfile("/root/.rclone_log/rclone_log.txt"):
  ### Rename log file and output settings.
  get_ipython().system_raw("mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt")
  with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
    f.write("Mode: " + Mode + \
            "\nCompare: " + Compare + \
            "\nSource: \"" + Source + \
            "\"\nDestination: \"" + Destination + \
            "\"\nTransfers: " + str(Transfers) + \
            "\nCheckers: " + str(Checkers) + \
            "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
            "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
            "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
            "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
            "\nDry-Run: " + str(Dry_Run) + \
            "\nOutput Log Level: " + Output_Log_File + \
            "\nExtra Arguments: \"" + Extra_Arguments + \
            "\"\nSync Moden: " + Sync_Mode + \
            "\nTrack Renames: " + str(Track_Renames) + \
            "\nDeduplicate Mode: " + Deduplicate_Mode + \
            "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))
  ### Compressing log file.
  get_ipython().system_raw("rm -f /root/rclone_log.zip && zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/ && rm -rf /root/.rclone_log/ && mkdir -p -m 666 /root/.rclone_log/")
  ### Send Log
  if os.path.isfile("/root/rclone_log.zip"):
    try:
      files.download("/root/rclone_log.zip")
      get_ipython().system_raw("rm -f /root/rclone_log.zip")
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Sending log to your browser...</h2><br></center>"))
    except:
      get_ipython().system_raw("mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip")
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can use file explorer to download the log file.</h2><br><img src=\"https://raw.githubusercontent.com/huseidon/RCloneColab/refs/heads/main/img/info.png\"><br></center>"))
  else:
    clear_output(wait=True)
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))

### Operation has been successfully completed.
if os.path.isfile("/usr/bin/rclone") == False:
  clear_output(wait=True)
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Please run the installation cell above first.</h2><br></center>"))
elif Mode == "Mount" and Source.strip() == "":
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ The Source field is empty.</h2><br></center>"))
elif Mode != "Config":
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))